In [1]:
# !kaggle competitions download -c sf-dl-car-classification
# !unzip sf-dl-car-classification.zip
# !unzip train.zip
# !unzip test.zip
# !rm sf-dl-car-classification.zip
# !rm train.zip
# !rm test.zip

In [2]:
import torch
import torch.nn as nn
from torch.nn.parallel import DataParallel
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

import cv2

from efficientnet_pytorch import EfficientNet

from torchsummary import summary

import albumentations
from albumentations.pytorch import ToTensorV2

%matplotlib widget

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
EPOCHS               = 10
BATCH_SIZE           = 16
LR                   = 1e-3
VAL_SPLIT            = 0.1

CLASS_NUM            = 10
IMG_SIZE             = 320 # 320, 520
IMG_CHANNELS         = 3
input_shape          = (IMG_SIZE-10, IMG_SIZE-10, IMG_CHANNELS)

PATH = './data'

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  

In [5]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, images_filepaths, transform=None, data='train'):
        self.images_filepaths = images_filepaths
        self.transform = transform
        self.data = data

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        img_info = self.images_filepaths.iloc[idx, :]
        label = img_info.Category
        if self.data == 'train':
            image_filepath = os.path.join('./data', self.data, str(label), img_info.Id)
        else:
            image_filepath = os.path.join('./data', self.data, img_info.Id)
            
        image = cv2.imread(image_filepath)
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        return image, label

In [6]:
AUGMENTATIONS_train = albumentations.Compose([
    albumentations.Resize(height=IMG_SIZE, width=IMG_SIZE),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(limit=10, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
    albumentations.CenterCrop(height=IMG_SIZE-10, width=IMG_SIZE-10),
    albumentations.RandomBrightnessContrast(brightness_limit=0.05, contrast_limit=0.05),
    albumentations.GaussianBlur(p=0.1),
    albumentations.HueSaturationValue(p=0.5),
    albumentations.RGBShift(p=0.5),
    albumentations.FancyPCA(alpha=0.1),
    albumentations.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

AUGMENTATIONS_test = albumentations.Compose([
    albumentations.Resize(height=IMG_SIZE, width=IMG_SIZE),
    albumentations.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

In [7]:
data_transforms = {
    'train': lambda image: AUGMENTATIONS_train(image=image),
    'val': lambda image: AUGMENTATIONS_test(image=image),
    'test': lambda image: AUGMENTATIONS_test(image=image),
}

In [8]:
data_dir = './data/'

image_locations = pd.read_csv("train.csv")
test_locations = pd.read_csv("sample-submission.csv")


image_locations = image_locations.sample(frac=1.)

In [9]:
train_dataset = MyDataset(images_filepaths=image_locations[:15000], transform=data_transforms['train'])
val_dataset = MyDataset(images_filepaths=image_locations[15000:], transform=data_transforms['val'])
test_dataset = MyDataset(images_filepaths=test_locations, transform=data_transforms['test'], data='test')

In [10]:
image_datasets = {'train': train_dataset,
                  'val': val_dataset,
                  'test': test_dataset}

dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset),
    'test': len(test_dataset)
}

In [11]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True)
              for x in ['train', 'val', 'test']}

In [12]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    plt.close()
#     plt.figure(figsize=(6, 4), dpi=150)
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.tight_layout()
    plt.show();


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [14]:
criterion = nn.CrossEntropyLoss()

In [15]:
class EfficientNet_b7(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=num_classes)
        
        self.classifier_layer = nn.Sequential(
            nn.BatchNorm1d(2560),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.5),
            nn.Linear(2560, num_classes)
        )
        
    def forward(self, inputs):
        x = self.model.extract_features(inputs)

        # Pooling and final linear layer
        x = self.model._avg_pooling(x)
        x = x.flatten(start_dim=1)
        x = self.classifier_layer(x)
        return x

In [16]:
model_ft = DataParallel(EfficientNet_b7(num_classes=CLASS_NUM), device_ids=[0, 1], output_device=0)
model_ft.to(device);

Loaded pretrained weights for efficientnet-b7


# Сперва сделаем Transfer Learning

In [17]:
for param in model_ft._modules['module'].model.parameters():
    param.requires_grad = False

In [18]:
# Observe that all parameters are being optimized
optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=1e-3)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.75)

In [19]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=EPOCHS)

Epoch 0/9
----------
train Loss: 1.5334 Acc: 0.4752
val Loss: 0.8561 Acc: 0.6970

Epoch 1/9
----------
train Loss: 1.3920 Acc: 0.5421
val Loss: 0.8094 Acc: 0.6952

Epoch 2/9
----------
train Loss: 1.4064 Acc: 0.5456
val Loss: 0.7734 Acc: 0.7255

Epoch 3/9
----------
train Loss: 1.3634 Acc: 0.5651
val Loss: 0.7455 Acc: 0.7273

Epoch 4/9
----------
train Loss: 1.3505 Acc: 0.5674
val Loss: 0.7681 Acc: 0.7094

Epoch 5/9
----------
train Loss: 1.3062 Acc: 0.5777
val Loss: 0.6996 Acc: 0.7362

Epoch 6/9
----------
train Loss: 1.2767 Acc: 0.5805
val Loss: 0.6927 Acc: 0.7611

Epoch 7/9
----------
train Loss: 1.2708 Acc: 0.5787
val Loss: 0.6796 Acc: 0.7558

Epoch 8/9
----------
train Loss: 1.2251 Acc: 0.5963
val Loss: 0.7163 Acc: 0.7540

Epoch 9/9
----------
train Loss: 1.2471 Acc: 0.5875
val Loss: 0.6976 Acc: 0.7469

Training complete in 66m 28s
Best val Acc: 0.761141


# А затем сделаем Fine Tuning

### Сперва с половиной слоёв, обученных на ImageNet

In [20]:
BATCH_SIZE           = 8

In [21]:
n_layers = len(list(model_ft._modules['module'].model.parameters()))

In [22]:
for param in list(model_ft._modules['module'].model.parameters())[n_layers // 2:]:
    param.requires_grad = True

In [23]:
optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=1e-4)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.75)

In [24]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=EPOCHS)

Epoch 0/9
----------
train Loss: 0.4663 Acc: 0.8481
val Loss: 0.1047 Acc: 0.9679

Epoch 1/9
----------
train Loss: 0.1862 Acc: 0.9415
val Loss: 0.0652 Acc: 0.9786

Epoch 2/9
----------
train Loss: 0.1330 Acc: 0.9547
val Loss: 0.0673 Acc: 0.9715

Epoch 3/9
----------
train Loss: 0.1140 Acc: 0.9632
val Loss: 0.0733 Acc: 0.9697

Epoch 4/9
----------
train Loss: 0.0921 Acc: 0.9688
val Loss: 0.0615 Acc: 0.9822

Epoch 5/9
----------
train Loss: 0.0612 Acc: 0.9791
val Loss: 0.0507 Acc: 0.9768

Epoch 6/9
----------
train Loss: 0.0530 Acc: 0.9813
val Loss: 0.0436 Acc: 0.9786

Epoch 7/9
----------
train Loss: 0.0501 Acc: 0.9833
val Loss: 0.0630 Acc: 0.9804

Epoch 8/9
----------
train Loss: 0.0415 Acc: 0.9875
val Loss: 0.0780 Acc: 0.9679

Epoch 9/9
----------
train Loss: 0.0325 Acc: 0.9889
val Loss: 0.0549 Acc: 0.9768

Training complete in 91m 59s
Best val Acc: 0.982175


In [29]:
# torch.save(model_ft.state_dict(), './pytorch_model.pth')

In [17]:
model_ft.load_state_dict(torch.load('./pytorch_model.pth', map_location='cpu'),)

<All keys matched successfully>

### А затем и на всей модели.

In [18]:
BATCH_SIZE           = 2

In [19]:
for param in list(model_ft._modules['module'].model.parameters()):
    param.requires_grad = True

In [20]:
optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=1e-5)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.75)

In [21]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=EPOCHS)

Epoch 0/9
----------
train Loss: 0.1500 Acc: 0.9513
val Loss: 0.0661 Acc: 0.9786

Epoch 1/9
----------
train Loss: 0.0897 Acc: 0.9695
val Loss: 0.0554 Acc: 0.9804

Epoch 2/9
----------
train Loss: 0.0722 Acc: 0.9748
val Loss: 0.0485 Acc: 0.9840

Epoch 3/9
----------
train Loss: 0.0637 Acc: 0.9779
val Loss: 0.0466 Acc: 0.9786

Epoch 4/9
----------
train Loss: 0.0539 Acc: 0.9807
val Loss: 0.0414 Acc: 0.9840

Epoch 5/9
----------
train Loss: 0.0475 Acc: 0.9823
val Loss: 0.0447 Acc: 0.9840

Epoch 6/9
----------
train Loss: 0.0412 Acc: 0.9869
val Loss: 0.0456 Acc: 0.9857

Epoch 7/9
----------
train Loss: 0.0402 Acc: 0.9869
val Loss: 0.0422 Acc: 0.9840

Epoch 8/9
----------
train Loss: 0.0320 Acc: 0.9886
val Loss: 0.0429 Acc: 0.9857

Epoch 9/9
----------
train Loss: 0.0331 Acc: 0.9893
val Loss: 0.0456 Acc: 0.9840

Training complete in 91m 35s
Best val Acc: 0.985740
